# Statistics in Python

In this section, we will cover how you can use Python to do some statistics. There are many packages to do so, but we will focus on four:

- [pandas](https://pandas.pydata.org/)
- [scipy's stats module](https://docs.scipy.org/doc/scipy/reference/stats.html)
- [statsmodels](http://www.statsmodels.org/stable/index.html)
- [seaborn](seaborn.pydata.org).

This notebook is strongly based on the [scipy-lectures.org](http://www.scipy-lectures.org/packages/statistics/index.html) section about statistics.

# Data representation and interaction

## Data as a table

The setting that we consider for statistical analysis is that of multiple *observations* or *samples* described by a set of different *attributes* or *features*. The data can than be seen as a 2D table, or matrix, with columns giving the different attributes of the data, and rows the observations. For instance, the data contained in `data/brain_size.csv`:

In [ ]:
!head data/brain_size.csv

## The pandas data-frame

### Creating dataframes: reading data files or converting arrays
 
#### Reading from a CSV file
Using the above CSV file that gives observations of brain size and weight and IQ (Willerman et al. 1991), the data are a mixture of numerical and categorical values::

In [ ]:
import pandas
data = pandas.read_csv('data/brain_size.csv', sep=';', na_values=".")
data

#### Creating from arrays
A `pandas.DataFrame` can also be seen as a dictionary of 1D 'series', eg arrays or lists. If we have 3 ``numpy`` arrays:

In [ ]:
import numpy as np
t = np.linspace(-6, 6, 20)
sin_t = np.sin(t)
cos_t = np.cos(t)

We can expose them as a `pandas.DataFrame`:

In [ ]:
pandas.DataFrame({'t': t, 'sin': sin_t, 'cos': cos_t}).head()

**Other inputs**: [pandas](http://pandas.pydata.org) can input data from SQL, excel files, or other formats. See the [pandas documentation](http://pandas.pydata.org).

### Manipulating data

`data` is a `pandas.DataFrame`, that resembles R's dataframe:

In [ ]:
data.shape    # 40 rows and 8 columns

In [ ]:
data.columns  # It has columns

In [ ]:
print(data['Hair'].head())  # Columns can be addressed by name

In [ ]:
# Simpler selector
data[data['Hair'] == 'light']['VIQ'].mean()

**Note:** For a quick view on a large dataframe, use its `describe` `pandas.DataFrame.describe`.

In [ ]:
data.describe()

In [ ]:
# Frequency count for a given column
data['Height'].value_counts()

In [ ]:
# Dummy-code # of hair color (i.e., get N-binary columns)
pandas.get_dummies(data['Hair'])[:15]

#### The [split-apply-combine](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf) pattern
* A very common data processing strategy is to...
    * Split the dataset into groups
    * Apply some operation(s) to each group
    * (Optionally) combine back into one dataset

Pandas provides powerful and fast tools for this. For example the `groupby` function.

**groupby**: splitting a dataframe on values of categorical variables:

In [ ]:
groupby_hair = data.groupby('Hair')
for hair, value in groupby_hair['VIQ']:
     print((hair, value.mean()))

`groupby_hair` is a powerful object that exposes many operations on the resulting group of dataframes:

In [ ]:
groupby_hair.mean()

### Exercise 1

* What is the mean value for VIQ for the full population?
* How many dark/light haired people were included in this study?
* What is the average value of MRI counts expressed in log units, for people with dark and light hair?

In [ ]:
data['VIQ'].mean()

In [ ]:
groupby_hair['Hair'].count()

In [ ]:
np.log(groupby_hair.MRI_Count.mean())

In [ ]:
# Create solution here

### Plotting data

Pandas comes with some plotting tools (`pandas.tools.plotting`, using
matplotlib behind the scene) to display statistics of the data in
dataframes.

For example, let's use `boxplot` (in this case even `groupby_hair.boxplot`) to better understand the structure of the data.

In [ ]:
%matplotlib inline
groupby_hair.boxplot(column=['FSIQ', 'VIQ', 'PIQ']);

#### Scatter matrices

In [ ]:
pandas.plotting.scatter_matrix(data[['Weight', 'Height', 'FSIQ']]);

# `scipy.stats` - Hypothesis testing: comparing two groups

For simple [statistical tests](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing), we will use the `scipy.stats` sub-module of [`scipy`](http://docs.scipy.org/doc/).

In [ ]:
from scipy import stats

## Student's t-test: the simplest statistical test

### 1-sample t-test: testing the value of a population mean

`scipy.stats.ttest_1samp` tests if the population mean of data is likely to be equal to a given value (technically if observations are drawn from a Gaussian distributions of given population mean). It returns the [T statistic](https://en.wikipedia.org/wiki/Student%27s_t-test), and the [p-value](https://en.wikipedia.org/wiki/P-value) (see the function's help):

In [ ]:
stats.ttest_1samp(data['VIQ'], 100)

With a p-value of 10^-28 we can claim that the population mean for the IQ (VIQ measure) is not 0.

### 2-sample t-test: testing for difference across populations

We have seen above that the mean VIQ in the dark hair and light hair populations
were different. To test if this is significant, we do a 2-sample t-test
with `scipy.stats.ttest_ind`:

In [ ]:
light_viq = data[data['Hair'] == 'light']['VIQ']
dark_viq = data[data['Hair'] == 'dark']['VIQ']
stats.ttest_ind(light_viq, dark_viq)

## Paired tests: repeated measurements on the same indivuals

PIQ, VIQ, and FSIQ give 3 measures of IQ. Let us test if FISQ and PIQ are significantly different. We can use a 2 sample test:

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

The problem with this approach is that it forgets that there are links
between observations: FSIQ and PIQ are measured on the same individuals.

Thus the variance due to inter-subject variability is confounding, and
can be removed, using a "paired test", or ["repeated measures test"](https://en.wikipedia.org/wiki/Repeated_measures_design):

In [ ]:
stats.ttest_rel(data['FSIQ'], data['PIQ'])

This is equivalent to a 1-sample test on the difference::

In [ ]:
stats.ttest_1samp(data['FSIQ'] - data['PIQ'], 0)

T-tests assume Gaussian errors. We can use a [Wilcoxon signed-rank test](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test), that relaxes this assumption:

In [ ]:
stats.wilcoxon(data['FSIQ'], data['PIQ'])

**Note:** The corresponding test in the non paired case is the [Mann–Whitney U test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U), `scipy.stats.mannwhitneyu`.

### Exercise 2

* Test the difference between weights in people with dark and light hair.
* Use non-parametric statistics to test the difference between VIQ in people with dark and light hair.

In [ ]:
light_weight = data[data['Hair'] == 'light']['Weight']
dark_weight = data[data['Hair'] == 'dark']['Weight']
stats.ttest_ind(light_weight, dark_weight, nan_policy='omit')

In [ ]:
stats.mannwhitneyu(light_viq, dark_viq)

**Conclusion**: we find that the data does not support the hypothesis that people with dark and light hair have different VIQ.

In [ ]:
# Create solution here

# `statsmodels` - use "formulas" to specify statistical models in Python

Use `statsmodels` to perform linear models, multiple factors or analysis of variance.


## A simple linear regression

Given two set of observations, `x` and `y`, we want to test the hypothesis that `y` is a linear function of `x`.

In other terms:

$y = x * coef + intercept + e$

where $e$ is observation noise. We will use the [statsmodels](http://statsmodels.sourceforge.net) module to:

1. Fit a linear model. We will use the simplest strategy, [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) (OLS).
2. Test that $coef$ is non zero.

First, we generate simulated data according to the model:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(-5, 5, 20)
np.random.seed(1)

# normal distributed noise
y = -5 + 3*x + 4 * np.random.normal(size=x.shape)

# Create a data frame containing all the relevant variables
data = pandas.DataFrame({'x': x, 'y': y})

plt.plot(x, y, 'o');

Then we specify an OLS model and fit it:

In [ ]:
from statsmodels.formula.api import ols
model = ols("y ~ x", data).fit()

**Note:** For more about "formulas" for statistics in Python, see the [statsmodels documentation](http://statsmodels.sourceforge.net/stable/example_formulas.html).

We can inspect the various statistics derived from the fit::

In [ ]:
print(model.summary())

### Terminology

Statsmodels uses a statistical terminology: the `y` variable in statsmodels is called *endogenous* while the `x` variable is called *exogenous*. This is discussed in more detail [here](http://statsmodels.sourceforge.net/devel/endog_exog.html). To simplify, `y` (endogenous) is the value you are trying to predict, while `x` (exogenous) represents the features you are using to make the prediction.

### Exercise 3

Retrieve the estimated parameters from the model above.  
**Hint**: use tab-completion to find the relevant attribute.

In [ ]:
model.params

In [ ]:
# Create solution here

## Categorical variables: comparing groups or multiple categories

Let us go back the data on brain size:

In [ ]:
data = pandas.read_csv('data/brain_size.csv', sep=';', na_values=".")

We can write a comparison between IQ of people with dark and light hair using a linear model:

In [ ]:
model = ols("VIQ ~ Hair + 1", data).fit()
print(model.summary())

### Tips on specifying model
 
***Forcing categorical*** - the 'Hair' is automatically detected as a categorical variable, and thus each of its different values is treated as different entities.

An integer column can be forced to be treated as categorical using:

```python
model = ols('VIQ ~ C(Hair)', data).fit()
```

***Intercept***: We can remove the intercept using `- 1` in the formula, or force the use of an intercept using `+ 1`.

### Link to t-tests between different FSIQ and PIQ

To compare different types of IQ, we need to create a "long-form" table, listing IQs, where the type of IQ is indicated by a categorical variable:

In [ ]:
data_fisq = pandas.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pandas.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pandas.concat((data_fisq, data_piq))
print(data_long[::8])

In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())

We can see that we retrieve the same values for t-test and corresponding p-values for the effect of the type of IQ than the previous t-test:

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

## Multiple Regression: including multiple factors

Consider a linear model explaining a variable `z` (the dependent
variable) with 2 variables `x` and `y`:

$z = x \, c_1 + y \, c_2 + i + e$

Such a model can be seen in 3D as fitting a plane to a cloud of (`x`,
`y`, `z`) points (see the following figure).

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
x = np.linspace(-5, 5, 21)

# We generate a 2D grid
X, Y = np.meshgrid(x, x)

# To get reproducable values, provide a seed value
np.random.seed(1)

# Z is the elevation of this 2D grid
Z = -5 + 3*X - 0.5*Y + 8 * np.random.normal(size=X.shape)

# Plot the data
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=plt.cm.coolwarm,
                       rstride=1, cstride=1)
ax.view_init(20, -120)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

### Example: the iris data (`data/iris.csv`)

Sepal and petal size tend to be related: bigger flowers are bigger! But is there, in addition, a systematic effect of species?

In [ ]:
from pandas.plotting import scatter_matrix

#Load the data
data = pandas.read_csv('data/iris.csv')

# Express the names as categories
categories = pandas.Categorical(data['Species'])

# The parameter 'c' is passed to plt.scatter and will control the color
scatter_matrix(data, c=categories.codes, marker='o')

# Plot figure
fig.suptitle("blue: setosa, green: versicolor, red: virginica", size=13)
plt.show()

In [ ]:
data = pandas.read_csv('data/iris.csv')
model = ols('SepalWidth ~ Species + PetalLength', data).fit()
print(model.summary())

## Post-hoc hypothesis testing: analysis of variance (ANOVA)

In the above iris example, we wish to test if the petal length is different between versicolor and virginica, after removing the effect of sepal width. This can be formulated as testing the difference between the coefficient associated to versicolor and virginica in the linear model estimated above (it is an Analysis of Variance, [ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance). For this, we write a **vector of 'contrast'** on the parameters estimated: we want to test ``"name[T.versicolor] - name[T.virginica]"``, with an [F-test](https://en.wikipedia.org/wiki/F-test):

In [ ]:
print(model.f_test([0, 1, -1, 0]))

Is this difference significant?

### Exercise 4

Going back to the brain size + IQ data, test if the VIQ of people with dark and light hair are different after removing the effect of brain size, height, and weight.

In [ ]:
data = pandas.read_csv('data/brain_size.csv', sep=';', na_values=".")
model = ols("VIQ ~ Hair + Height + Weight + MRI_Count", data).fit()
print(model.summary())

In [ ]:
# Create solution here

# `seaborn` - use visualization for statistical exploration

[Seaborn](http://stanford.edu/~mwaskom/software/seaborn/) combines simple statistical fits with plotting on pandas dataframes.

Let us consider a data giving wages and many other personal information on 500 individuals ([Berndt, ER. The Practice of Econometrics. 1991. NY:Addison-Wesley](http://lib.stat.cmu.edu/datasets/CPS_85_Wages)).

In [ ]:
import pandas
data = pandas.read_csv('data/wages.csv', sep=',')
data.head()

## Pairplot: scatter matrices

We can easily have an intuition on the interactions between continuous variables using `seaborn.pairplot` to display a scatter matrix:

In [ ]:
import seaborn
seaborn.set()
seaborn.pairplot(data, vars=['WAGE', 'AGE', 'EDUCATION'], kind='reg')

Categorical variables can be plotted as the hue:

In [ ]:
seaborn.pairplot(data, vars=['WAGE', 'AGE', 'EDUCATION'], kind='reg', hue='HAIR')

## lmplot: plotting a univariate regression

A regression capturing the relation between one variable and another, e.g. wage and eduction, can be plotted using `seaborn.lmplot`:

In [ ]:
seaborn.lmplot(y='WAGE', x='EDUCATION', data=data)

### Robust regression
Given that, in the above plot, there seems to be a couple of data points that are outside of the main cloud to the right, they might be outliers, not representative of the population, but driving the regression.

To compute a regression that is less sensitive to outliers, one must use a [robust model](https://en.wikipedia.org/wiki/Robust_statistics). This is done in seaborn using ``robust=True`` in the plotting functions, or in statsmodels by replacing the use of the OLS by a "Robust Linear Model", `statsmodels.formula.api.rlm`.

# Testing for interactions

Do wages increase more with education for people with dark hair than with light hair?

In [ ]:
seaborn.lmplot(y='WAGE', x='EDUCATION', hue='HAIR', data=data)

The plot above is made of two different fits. We need to formulate a single model that tests for a variance of slope across the population. This is done via an ["interaction"](http://statsmodels.sourceforge.net/devel/example_formulas.html#multiplicative-interactions).

In [ ]:
from statsmodels.formula.api import ols
result = ols(formula='WAGE ~ EDUCATION + HAIR + EDUCATION * HAIR', data=data).fit()
print(result.summary())

Can we conclude that education benefits people with dark hair more than people with light hair?

# Take home messages

* Hypothesis testing and p-value give you the **significance** of an effect / difference

* **Formulas** (with categorical variables) enable you to express rich links in your data

* **Visualizing** your data and simple model fits matters!

* **Conditioning** (adding factors that can explain all or part of the variation) is an important modeling aspect that changes the interpretation.